In [45]:
#Importamos los conectores para Snowflake
import snowflake.connector as sf
from snowflake.connector import DictCursor
import pandas as pd 


In [46]:
#Conexion a Snowflake 
username   = 'Blockchainassets'
mypassword = 'BonneNouvelle2020'
myaccount  = 'iw74759.eu-central-1'
warehouse  = 'COMPUTE_WH'
database   = 'DEMO_DB'

conn = sf.connect(user=username, password = mypassword, account = myaccount)
#Función que permite realizar la conexion a la nube de Snowflake y realizar procedimientos sql
def execute_query(connection, query):
    cursor = connection.cursor()
    cursor.execute(query)
    cursor.close()
    
#Conexión al WareHouse y el nombre de la Base de Datos a usar
try:
    sql = 'use {}'.format(database)
    execute_query(conn,sql)
    
    sql = 'use warehouse {}'.format(warehouse)
    execute_query(conn,sql)
    
    try:   
        sql = 'alter warehouse {} resume'.format(warehouse)
        execute_query(conn,sql)
    except: 
        pass
except Exception as e:
    print(e)


In [ ]:
#Extraemos datos del archivo CSV 
raw = pd.read_csv('10Y-us-rate-data.csv')
raw.DGS10 = pd.to_numeric(raw.DGS10,errors ='coerce')

In [ ]:
#Insertar datos
try:
    #--- Comienza una función para encapsular los procesos de insertar
    def function_insert():
        #Comienza la transacción
        startbeging = "begin name t4;"
        #Ejecutamos el Begin antes de realizar los insert into
        execute_query(conn, startbeging)
        
        #--- Comienza For (Ciclo para insertar 50 registros)
        #Iterar nuestro archivo csv  - insertando 50 registros. 
        for index in range(50):
            sqldate  = raw['DATE'].loc[index]
            sqldgs10 = raw['DGS10'].loc[index]
            sql = "insert into US_RATE_DATA (DATE, DGS10) values ('{}','{}')".format(sqldate, sqldgs10)
            execute_query(conn, sql) 
        #--- Termina el FOR
        
        #Al terminar todo los insert ejecutamos el Commit para finalizar la transacción
        commit = "commit;"
        execute_query(conn,commit)
        print("Operación Exitosa")
    #--- Termina la función_insert 
    
    #Se ejecuta la función y realiza el proceso
    function_insert()
except:
    #En caso de una incidencia/riesgo se ejecuta el Rollback y se ejecutará de nuevo la función_insert
    rollback = "rollback;"
    execute_query(conn, rollback)
    print("Error, Operación Cancelada / Rollback / Se ejecutara de nuevo")
    function_insert()


In [ ]:
#Realizar una consulta SQL y convertilo en DataFrame
try:
    sql = "select count(*) as Total_Registros from US_RATE_DATA"
    cursor = conn.cursor()
    cursor.execute(sql)
 
    for c in cursor: 
        print(c)
    cursor.close()
    #Convertir la consulta en dataframe para manipular sus datos
    #df = pd.read_sql(sql, conn)
    #print(df.iloc[0,0])
    #df.info()
    #print(df.head())
    #print(df.count())
except Exception as e:
    print(e)